In [141]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms
import cv2
from torch.utils.data import Dataset, DataLoader

PATH = "kaggle_3m"
IMAGE_SIZE = (256, 256)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

paths = []

for dirname in os.listdir(PATH):
    if os.path.isdir(os.path.join(PATH, dirname)):
        
        for filename in os.listdir(os.path.join(PATH, dirname)):
            if filename.endswith('.tif'):
                paths.append(PATH+'/'+dirname+'/'+filename)


def data_frame(data):
    images = list(filter(lambda x: not x.endswith('mask.tif'), data))
    images.sort(key=lambda x: int(x.rsplit('_', 3)[-1][:-4]))
    images.sort(key=lambda x: int(x.rsplit('_', 3)[-2]))
    
    IDs = list(map(lambda x: x.rsplit('/', 3)[-1][:-4], images))

    masks = list(filter(lambda x: x.endswith('mask.tif'), data))
    masks.sort(key=lambda x: int(x.rsplit('_', 3)[-2]))
    masks.sort(key=lambda x: int(x.rsplit('_', 3)[-3]))

    pixels = lambda x: Image.open(x)
    largest_pixel = lambda y: np.max(pixels(y))
    diagnotic_function = lambda z: 1 if largest_pixel(z) > 0 else 0
    diagnoses = list(map(lambda x: diagnotic_function(x), masks))

    data_df = pd.DataFrame({'ID': IDs, 'Image': images, 'Mask': masks, 'Diagnosis': diagnoses})
    
    train_index, val_index = train_test_split(data_df.index.values.tolist(), test_size=0.19, random_state=42)
    val_index, test_index = train_test_split(val_index, test_size=0.12, random_state=42)
    
    # Making train, test, and validation dataframes
    train_df, val_df, test_df = data_df.iloc[train_index], data_df.iloc[val_index], data_df.iloc[test_index]
    
    return train_df, val_df, test_df
    
# Making the dataframes
train_df, val_df, test_df = data_frame(paths)

print(len(train_df), len(val_df), len(test_df))

print(train_df.head())

def tensor_from_path(path):
    arr = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    arr = cv2.resize(arr, IMAGE_SIZE)
    arr = arr / 255
    if len(arr.shape) == 3:
        tensor = torch.tensor(arr).permute(2,0,1)
    elif len(arr.shape) == 2:
        tensor = torch.tensor(arr).unsqueeze(0)
    else:
        raise ValueError(f"Expected data shape to be (..., ..., 3) or (..., ...) , but got {arr.shape}")
    return tensor

class data(Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df        
        self.images = self.df.loc[:,'Image'].values
        self.masks = self.df.loc[:,'Mask'].values
        self.diagnosis = self.df.loc[:,'Diagnosis'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        im_path = self.images[idx]
        msk_path= self.masks[idx]
        diagnosis = self.diagnosis[idx]
        self.im_tensor = tensor_from_path(im_path)
        self.msk_tensor= tensor_from_path(msk_path)
        return self.im_tensor.to(device), self.msk_tensor.to(device),diagnosis

train_ds = data(train_df)
test_ds = data(test_df)

train_dl = DataLoader(train_ds, batch_size=32)
test_dl = DataLoader(test_ds, batch_size=32)

cpu
3182 657 90
                            ID  \
812   TCGA_DU_7302_19911203_33   
1550  TCGA_HT_7602_19951103_17   
2228  TCGA_DU_A5TR_19970726_31   
1108  TCGA_DU_5872_19950223_26   
3728  TCGA_FG_7643_20021104_17   

                                                  Image  \
812   kaggle_3m/TCGA_DU_7302_19911203/TCGA_DU_7302_1...   
1550  kaggle_3m/TCGA_HT_7602_19951103/TCGA_HT_7602_1...   
2228  kaggle_3m/TCGA_DU_A5TR_19970726/TCGA_DU_A5TR_1...   
1108  kaggle_3m/TCGA_DU_5872_19950223/TCGA_DU_5872_1...   
3728  kaggle_3m/TCGA_FG_7643_20021104/TCGA_FG_7643_2...   

                                                   Mask  Diagnosis  
812   kaggle_3m/TCGA_DU_7302_19911203/TCGA_DU_7302_1...          0  
1550  kaggle_3m/TCGA_HT_7602_19951103/TCGA_HT_7602_1...          0  
2228  kaggle_3m/TCGA_DU_A5TR_19970726/TCGA_DU_A5TR_1...          0  
1108  kaggle_3m/TCGA_DU_5872_19950223/TCGA_DU_5872_1...          0  
3728  kaggle_3m/TCGA_FG_7643_20021104/TCGA_FG_7643_2...          0  


In [142]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [143]:
import torch
import torch.nn as nn

class BottleNeckBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, downsample=None):
        super().__init__()

        base_width = 64
        width = int(out_channels * (base_width / 64.)) * 1

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=width, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=width)
        self.conv2 = nn.Conv2d(in_channels=width, out_channels=width, kernel_size=kernel_size, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=width)
        self.conv3 = nn.Conv2d(in_channels=width, out_channels=out_channels * self.expansion, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(num_features=out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        print(f"Input: {x.shape}")

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        print(f"After conv1: {out.shape}")

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        print(f"After conv2: {out.shape}")

        out = self.conv3(out)
        out = self.bn3(out)
        print(f"After conv3: {out.shape}")

        if self.downsample is not None:
            identity = self.downsample(x)
            print(f"Downsampled identity: {identity.shape}")

        out += identity
        out = self.relu(out)
        print(f"Output: {out.shape}")

        return out


In [144]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes):
        super().__init__()

        self.in_channels = 64

        # resnet stem
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(num_features=self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # res-blocks
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # classifier block
        self.adppool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(in_features=512 * block.expansion, out_features=num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None

        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels=self.in_channels, out_channels=out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(num_features=out_channels * block.expansion)
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion

        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        print(f"Input: {x.shape}")
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        print(f"After conv1: {x.shape}")
        x = self.maxpool(x)
        print(f"After maxpool: {x.shape}")

        x = self.layer1(x)
        print(f"After layer1: {x.shape}")
        x = self.layer2(x)
        print(f"After layer2: {x.shape}")
        x = self.layer3(x)
        print(f"After layer3: {x.shape}")
        x = self.layer4(x)
        print(f"After layer4: {x.shape}")

        x = self.adppool(x)
        print(f"After adppool: {x.shape}")
        x = torch.flatten(x, 1)
        print(f"After flatten: {x.shape}")

        return self.classifier(x)


In [151]:
resnet34 = ResNet(block=BottleNeckBlock, layers=[3,4,6,3], num_classes=1).to(device)
from torchinfo import summary

print(summary(resnet34, input_shape=(4, 3, 256, 256)))

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─BottleNeckBlock: 2-1              --
│    │    └─Conv2d: 3-1                  4,096
│    │    └─BatchNorm2d: 3-2             128
│    │    └─Conv2d: 3-3                  4,096
│    │    └─BatchNorm2d: 3-4             128
│    │    └─Conv2d: 3-5                  16,384
│    │    └─BatchNorm2d: 3-6             512
│    │    └─ReLU: 3-7                    --
│    └─BottleNeckBlock: 2-2              --
│    │    └─Conv2d: 3-8                  16,384
│    │    └─BatchNorm2d: 3-9             128
│    │    └─Conv2d: 3-10                 36,864
│    │    └─BatchNorm2d: 3-11            128
│    │    └─Conv2d: 3-12                 16,384
│    │    └─BatchNorm2d: 3-13           

In [156]:
import torch.optim as optim
resnet34 = ResNet18()
criterion = nn.BCELoss()
optimizer = optim.Adam(resnet34.parameters(), lr=0.001)

dataloader = torch.utils.data.DataLoader(train_ds, batch_size=4, shuffle=True)

for epoch in range(7):
    correct = 0
    total = 0
    current_predicted = 0
    current_total = 0
    for i, data in enumerate(dataloader, 0):

        inputs, _,labels = data
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)

        optimizer.zero_grad()

        outputs = resnet34(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        predicted = torch.round(outputs.data) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        current_total += labels.size(0)
        current_predicted += (predicted == labels).sum().item()
        if i % 100 == 0:
            print('epoch',epoch)
            print('Current accaracy: %d %%' % (100 * current_predicted / current_total))
            print('Accuracy of the network on the train images: %d %%' % (100 * correct / total))
            current_predicted = 0
            current_total = 0


print('Finished Training')


epoch 0
Current accaracy: 25 %
Accuracy of the network on the train images: 25 %
epoch 0
Current accaracy: 61 %
Accuracy of the network on the train images: 60 %
epoch 0
Current accaracy: 64 %
Accuracy of the network on the train images: 62 %
epoch 0
Current accaracy: 63 %
Accuracy of the network on the train images: 62 %
epoch 0
Current accaracy: 70 %
Accuracy of the network on the train images: 64 %
epoch 0
Current accaracy: 63 %
Accuracy of the network on the train images: 64 %
epoch 0
Current accaracy: 62 %
Accuracy of the network on the train images: 64 %
epoch 0
Current accaracy: 62 %
Accuracy of the network on the train images: 63 %
epoch 1
Current accaracy: 0 %
Accuracy of the network on the train images: 0 %
epoch 1
Current accaracy: 68 %
Accuracy of the network on the train images: 67 %
epoch 1
Current accaracy: 62 %
Accuracy of the network on the train images: 64 %
epoch 1
Current accaracy: 70 %
Accuracy of the network on the train images: 66 %
epoch 1
Current accaracy: 67 %

In [155]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import models

net = models.resnet18()
print(net)

class ResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net18 = models.resnet18()
        self.fc = nn.Linear(1000,1) 

    def forward(self, x):
        x = self.net18(x)
        x = self.fc(x)
        return torch.sigmoid(x)
    



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  